<a href="https://colab.research.google.com/github/leonardocerliani/Jorge4PubMed/blob/main/create_citations_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jorge4PubMed companion notebook

**NB: this notebook is meant to be run with a [local python kernel/runtime](https://research.google.com/colaboratory/local-runtimes.html)**.

If you want to use the Colab runtime, you will need to modify the code below so that it operates on an `md_out` folder hosted on the mounted Google Drive.

[GitHub repo](https://github.com/leonardocerliani/Jorge4PubMed)

![](https://raw.githubusercontent.com/leonardocerliani/Jorge4PubMed/refs/heads/main/imgs/jorge.png)

In [18]:
# Filename for the db of citations db
# (contains filename, title, PMID, full_citation, weblink)
db_filename = "citations.db"
logfile = "log_citation_db_creation.log"

# Set your email here to use NCBI Entrez API
email_for_entrez_service = "my_email@gmail.com"
limit_esearch_per_second = 3 # Use n > 3 ONLY if you have a PubMed api key

# Location of the markdown files extracted from the papers' pdf
md_folder = 'md_out'
n_lines = 20


# Batch size for extracting title using gpt-4o
llm_extract_titles_batch_size = 10


# Set interactive to True if you want to manually run one cell at the time to
# to inspect the process. Set to False if you just want to run the main()
# at the end of the notebook.
interactive_mode = True

# ---------------- MODIFY THE CODE BELOW AT YOUR OWN RISK ----------------------

# Functions


## Libraries

In [19]:
import time
import os
import sqlite3
import pandas as pd
from pprint import pprint
import concurrent.futures
from IPython.display import display

from Bio import Entrez
Entrez.email = email_for_entrez_service

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI()

import logging
logging.basicConfig(filename=logfile, level=logging.INFO, format='%(asctime)s - %(message)s')

import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

## Cosine similarity to verify titles

In [21]:
%%time

# Cosine similarity between to sentences
model = SentenceTransformer('all-MiniLM-L6-v2')

def batch_sentence_similarity(list1, list2):
    assert len(list1) == len(list2), "Lists must be of equal length"
    embeddings1 = model.encode(list1)
    embeddings2 = model.encode(list2)
    similarities = np.array([
        cosine_similarity([emb1], [emb2])[0][0]
        for emb1, emb2 in zip(embeddings1, embeddings2)
    ])
    return similarities


# # Example usage
# list1 = ["The dog sat on the mat.","A quick brown fox jumps over the lazy dog.","I love machine learning.","The weather is nice today."]
# list2 = ["There is a cat on the mat.","The fox jumped over a lazy dog quickly.","Machine learning is amazing.","It's a beautiful day outside."]

# similarities = batch_sentence_similarity(list1,list2)
# similarities


CPU times: user 91.2 ms, sys: 70.7 ms, total: 162 ms
Wall time: 1.61 s


## Create citations.db and open the connection

In [22]:
# Create citations.db if not already existing
def create_citations_db_if_not_existing(conn):
    cursor = conn.cursor()
    print(f"Database '{db_filename}' loaded or created successfully.")

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS citations (
        title TEXT,
        PMID TEXT,
        filename TEXT,
        full_citation TEXT,
        weblink TEXT,
        Pubmed_title TEXT
    )
    """)
    pprint(cursor.execute("PRAGMA table_info(citations)").fetchall())


if interactive_mode:
  conn = sqlite3.connect(db_filename)
  create_citations_db_if_not_existing(conn)

Database 'citations.db' loaded or created successfully.
[(0, 'title', 'TEXT', 0, None, 0),
 (1, 'PMID', 'TEXT', 0, None, 0),
 (2, 'filename', 'TEXT', 0, None, 0),
 (3, 'full_citation', 'TEXT', 0, None, 0),
 (4, 'weblink', 'TEXT', 0, None, 0),
 (5, 'Pubmed_title', 'TEXT', 0, None, 0)]


## Get new papers filename

In [23]:
# Get new papers filename
def get_new_papers_filename(md_folder, n_lines, conn):
    new_papers = []
    cursor = conn.cursor()

    cursor.execute("SELECT filename FROM citations")
    existing_filenames = [row[0] for row in cursor.fetchall()]

    # Log duplicated filenames found in md_folder that already exist in DB
    for filename in os.listdir(md_folder):
        if filename.endswith('.md'):
            if filename in existing_filenames:
                logging.warning(f"Duplicate filename detected: '{filename}' already exists in the database.")

    print(f"Scanning folder '{md_folder}' for new markdown files...")

    for filename in os.listdir(md_folder):
        if filename.endswith('.md'):
            path = os.path.join(md_folder, filename)

            with open(path, 'r', encoding='utf-8') as f:
                lines = []
                for _ in range(n_lines):
                    try:
                        lines.append(next(f).strip())
                    except StopIteration:
                        break
                multiline_string = "\n".join(lines)

            if filename not in existing_filenames:
                new_papers.append({
                    "title": None,
                    "PMID": None,
                    "filename": filename,
                    "first_lines": multiline_string
                })

    print(f"Found {len(new_papers)} new markdown files to process.")
    return new_papers

# Usage
if interactive_mode:
  new_papers = get_new_papers_filename(md_folder, n_lines, conn)

Scanning folder 'md_out' for new markdown files...
Found 14 new markdown files to process.


## Extract titles from the md using gpt-4.1-nano

In [24]:
%%time

# Extract titles from the md using gpt-4.1-nano
def extract_one_title(content):
    extract_title_parsing_instructions = """
    The following are the first lines of a scientific paper in markdown format.
    You task is to extract the title and return **ONLY** the title and nothing else.

    - Ignore any markdown formatting, headings, or special characters (e.g., `#`, `**`, etc.).
    - Try to correct spelling mistakes for common english words.
    - Do not correct anything in technical terms.

    It might be helpful to know that:
    - the very first line can be a collection heading (e.g. "Opinions", "Original Research", "Review paper", "Sort Communication") instead of the title.
    - the title is usually just before the author's list
    """

    response = client.responses.create(
        model='gpt-4.1-nano',
        input=content,
        instructions=extract_title_parsing_instructions,
        temperature=0.3
    )
    return response.output_text


def llm_extract_titles(new_papers, batch_size=llm_extract_titles_batch_size):
    total = len(new_papers)
    print(f"Extracting titles from {total} papers using GPT-4.1-nano in batches of {batch_size}...")

    for i in range(0, total, batch_size):
        batch = new_papers[i:i + batch_size]
        contents = [paper["first_lines"] for paper in batch]

        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = list(executor.map(extract_one_title, contents))

        for paper, title in zip(batch, results):
            paper["title"] = title
            del paper["first_lines"]

        print(f"Processed titles for papers {i+1} to {min(i+batch_size, total)} / {total}")


def log_duplicate_titles(new_papers, conn):
    cursor = conn.cursor()
    cursor.execute("SELECT title, filename FROM citations WHERE title IS NOT NULL")
    existing_records = cursor.fetchall()

    existing_titles = {}
    for title, filename in existing_records:
        if title in existing_titles:
            existing_titles[title].append(filename)
        else:
            existing_titles[title] = [filename]

    for paper in new_papers:
        title = paper['title']
        if title in existing_titles:
            existing_files = ', '.join(existing_titles[title])
            logging.warning(f"Duplicate title found: '{title}' from file '{paper['filename']}' matches existing files: {existing_files}")


if interactive_mode:
  llm_extract_titles(new_papers, batch_size=10)
  log_duplicate_titles(new_papers, conn)
  display(pd.DataFrame(new_papers))

Extracting titles from 14 papers using GPT-4.1-nano in batches of 10...
Processed titles for papers 1 to 10 / 14
Processed titles for papers 11 to 14 / 14


,title,PMID,filename
0,RISC Assembly Defects in the Drosophila RNAi M...,None,1009Tomari2004.md
1,Regulation of Constitutive GPR3 Signaling and ...,None,11Lowther2013.md
2,A Role for Calcium Release-Activated Current C...,None,3953Bertram1995.md
3,News and Views,None,114c.md
4,"Crucial Role of Type 1, but Not Type 3, Inosit...",None,1020.md
5,Spreading silence with Sid,None,1011van_Roessel2004.md
6,The functional universe of membrane contact sites,None,Prinz2019The_functional_universe_of_membrane.md
7,Cytoplasmic Ca2+ oscillations evoked by recept...,None,8.md
8,Therapeutic silencing of an endogenous gene by...,None,1003Soutschek2004.md
9,Animal virus replication and RNAi-mediated ant...,None,1000Lu2005.md


CPU times: user 81.2 ms, sys: 17 ms, total: 98.1 ms
Wall time: 3.89 s


## Initial esearch of PMID on Pubmed with lenient proximity (e.g. proximity = 2)

In [25]:
%%time

# Initial search of PMID on Pubmed esearch
def fetch_PMIDs_with_rate_limit(new_papers, proximity=2, limit_esearch_per_second=3):
    total = len(new_papers)
    processed = 0

    for paper in new_papers:
        title = paper["title"]
        try:
            handle = Entrez.esearch(db="pubmed", term=f'"{title}"[Title:~{proximity}]', retmax=1)
            record = Entrez.read(handle)
            handle.close()
            id_list = record.get("IdList")
            paper["PMID"] = id_list[0] if id_list else ''
        except Exception as e:
            logging.error(f"Error fetching PMID for title '{title}': {e}")
            paper["PMID"] = ''

        processed += 1
        print(f"Fetched PMID for {processed}/{total} papers", end="\r", flush=True)

        if processed % limit_esearch_per_second == 0:
            time.sleep(1)

    print()



if interactive_mode:
    proximity_initial_search = 2
    limit_esearch_per_second = 3 # Use n > 3 ONLY if you have a PubMed api key

    fetch_PMIDs_with_rate_limit(
        new_papers,
        proximity = proximity_initial_search,
        limit_esearch_per_second = limit_esearch_per_second
    )

    display(pd.DataFrame(new_papers).style)




,title,PMID,filename
0,RISC Assembly Defects in the Drosophila RNAi Mutant Armitage,15035985,1009Tomari2004.md
1,Regulation of Constitutive GPR3 Signaling and Surface Localization by GRK2 and b-arrestin-2 Overexpression in HEK293 Cells,,11Lowther2013.md
2,A Role for Calcium Release-Activated Current CRAC in Cholinergic Modulation of Electrical Activity in Pancreatic 3-Cells,,3953Bertram1995.md
3,News and Views,40397855,114c.md
4,"Crucial Role of Type 1, but Not Type 3, Inositol 1,4,5-Trisphosphate (IP3) Receptors in IP3-Induced Ca2 Release, Capacitative Ca2 Entry, and Proliferation of A7r5 Vascular Smooth Muscle Cells",,1020.md
5,Spreading silence with Sid,14759251,1011van_Roessel2004.md
6,The functional universe of membrane contact sites,31732717,Prinz2019The_functional_universe_of_membrane.md
7,"Cytoplasmic Ca2+ oscillations evoked by receptor stimulation, G-protein activation, internal application of inositol trisphosphate or Ca2+: simultaneous microfluorimetry and Ca2+ dependent Cl- current recording in single pancreatic acinar cells",1690123,8.md
8,Therapeutic silencing of an endogenous gene by systemic administration of modified siRNAs,15538359,1003Soutschek2004.md
9,Animal virus replication and RNAi-mediated antiviral silencing in Caenorhabditis elegans,16107851,1000Lu2005.md


CPU times: user 69.6 ms, sys: 26.2 ms, total: 95.8 ms
Wall time: 10.2 s


## Get the title corresponding to the found PMID and store it in a new key `Pubmed_title`

In [26]:
%%time

# Get the titles of the identified PMIDs using efetch
def get_pubmed_titles(new_papers, batch_size=100):
    papers_with_pmid = [paper for paper in new_papers if paper.get('PMID')]
    if not papers_with_pmid:
        print("No papers with PMIDs found for validation.")
        return

    print(f"Fetching Pubmed titles for {len(papers_with_pmid)} papers...")

    for i in range(0, len(papers_with_pmid), batch_size):
        batch = papers_with_pmid[i:i + batch_size]
        pmids_string = ",".join([p['PMID'] for p in batch])

        try:
            handle = Entrez.efetch(db="pubmed", id=pmids_string, retmode="xml")
            records = Entrez.read(handle)
            handle.close()

            for paper, article in zip(batch, records['PubmedArticle']):
                try:
                    fetched_title = article['MedlineCitation']['Article']['ArticleTitle']
                    paper['Pubmed_title'] = fetched_title  # Just store the title
                except Exception as e:
                    logging.error(f"Error fetching title for PMID {paper.get('PMID')}: {e}")
                    paper['Pubmed_title'] = ''

            print(f"Fetched Pubmed titles for papers {i+1} to {min(i+batch_size, len(papers_with_pmid))}")

        except Exception as e:
            logging.error(f"Entrez fetch error for batch starting at index {i}: {e}")

        print("Sleeping for 2 seconds to comply with Pubmed rate limit")
        time.sleep(2)



if interactive_mode:
  get_pubmed_titles(new_papers)
  display(pd.DataFrame(new_papers))

Fetching Pubmed titles for 11 papers...
Fetched Pubmed titles for papers 1 to 11
Sleeping for 2 seconds to comply with Pubmed rate limit


,title,PMID,filename,Pubmed_title
0,RISC Assembly Defects in the Drosophila RNAi M...,15035985,1009Tomari2004.md,RISC assembly defects in the Drosophila RNAi m...
1,Regulation of Constitutive GPR3 Signaling and ...,,11Lowther2013.md,NaN
2,A Role for Calcium Release-Activated Current C...,,3953Bertram1995.md,NaN
3,News and Views,40397855,114c.md,"Annals in Action: Your Newest Source for News,..."
4,"Crucial Role of Type 1, but Not Type 3, Inosit...",,1020.md,NaN
5,Spreading silence with Sid,14759251,1011van_Roessel2004.md,Spreading silence with Sid.
6,The functional universe of membrane contact sites,31732717,Prinz2019The_functional_universe_of_membrane.md,The functional universe of membrane contact si...
7,Cytoplasmic Ca2+ oscillations evoked by recept...,1690123,8.md,Cytoplasmic Ca2+ oscillations evoked by recept...
8,Therapeutic silencing of an endogenous gene by...,15538359,1003Soutschek2004.md,Therapeutic silencing of an endogenous gene by...
9,Animal virus replication and RNAi-mediated ant...,16107851,1000Lu2005.md,Animal virus replication and RNAi-mediated ant...


CPU times: user 56.7 ms, sys: 10.3 ms, total: 67 ms
Wall time: 2.69 s


## # Reject PMID where the similarity of title with Pubmed_title is < 0.94

In [28]:
%%time

# # Reject PMID where the similarity of title with Pubmed_title is < 0.94
# similarity_threshold_initial_search = 0.94

def compare_titles_embeddings(new_papers,similarity_threshold):
    # Filter papers where Pubmed_title exists and is non-empty
    filtered = [p for p in new_papers if p.get('Pubmed_title')]

    if not filtered:
        print("No papers with 'Pubmed_title' to compare.")
        return pd.DataFrame()

    titles = [p['title'] for p in filtered]
    pubmed_titles = [p['Pubmed_title'] for p in filtered]
    pmids = [p['PMID'] for p in filtered]

    similarities = batch_sentence_similarity(titles, pubmed_titles)

    # Update original papers based on similarity
    for paper, sim in zip(filtered, similarities):
      if sim < similarity_threshold:
          paper['PMID'] = ''

    df = pd.DataFrame({
        'title': [p['title'] for p in filtered],
        'Pubmed_title': [p.get('Pubmed_title', '') for p in filtered],
        'cosine_similarity': similarities,
        'PMID': [p['PMID'] for p in filtered]
    })

    return df


if interactive_mode:
  similarity_threshold_initial_search = 0.94
  df = compare_titles_embeddings(
      new_papers,
      similarity_threshold=similarity_threshold_initial_search
  )
  df.style
  display(pd.DataFrame(new_papers).style)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,title,PMID,filename,Pubmed_title
0,RISC Assembly Defects in the Drosophila RNAi Mutant Armitage,15035985,1009Tomari2004.md,RISC assembly defects in the Drosophila RNAi mutant armitage.
1,Regulation of Constitutive GPR3 Signaling and Surface Localization by GRK2 and b-arrestin-2 Overexpression in HEK293 Cells,,11Lowther2013.md,nan
2,A Role for Calcium Release-Activated Current CRAC in Cholinergic Modulation of Electrical Activity in Pancreatic 3-Cells,,3953Bertram1995.md,nan
3,News and Views,,114c.md,"Annals in Action: Your Newest Source for News, Views, and Reviews in Plastic Surgery."
4,"Crucial Role of Type 1, but Not Type 3, Inositol 1,4,5-Trisphosphate (IP3) Receptors in IP3-Induced Ca2 Release, Capacitative Ca2 Entry, and Proliferation of A7r5 Vascular Smooth Muscle Cells",,1020.md,nan
5,Spreading silence with Sid,14759251,1011van_Roessel2004.md,Spreading silence with Sid.
6,The functional universe of membrane contact sites,31732717,Prinz2019The_functional_universe_of_membrane.md,The functional universe of membrane contact sites.
7,"Cytoplasmic Ca2+ oscillations evoked by receptor stimulation, G-protein activation, internal application of inositol trisphosphate or Ca2+: simultaneous microfluorimetry and Ca2+ dependent Cl- current recording in single pancreatic acinar cells",1690123,8.md,"Cytoplasmic Ca2+ oscillations evoked by receptor stimulation, G-protein activation, internal application of inositol trisphosphate or Ca2+: simultaneous microfluorimetry and Ca2+ dependent Cl- current recording in single pancreatic acinar cells."
8,Therapeutic silencing of an endogenous gene by systemic administration of modified siRNAs,15538359,1003Soutschek2004.md,Therapeutic silencing of an endogenous gene by systemic administration of modified siRNAs.
9,Animal virus replication and RNAi-mediated antiviral silencing in Caenorhabditis elegans,16107851,1000Lu2005.md,Animal virus replication and RNAi-mediated antiviral silencing in Caenorhabditis elegans.


CPU times: user 36.4 ms, sys: 32.9 ms, total: 69.2 ms
Wall time: 95.2 ms


## Deep search of the title using a window_of_words and strict proximity = 0

In [29]:
# Deep search using window_of_words and proximity
def word_window_proximity_search(new_papers, window_of_words_size=5, proximity_deep_search=0, limit_esearch_per_second = 3):
    print(f"Starting word window proximity search for missing PMIDs (window size = {window_of_words_size}, proximity = {proximity_deep_search})...")

    for idx, paper in enumerate(new_papers, 1):
        if paper.get("PMID", '') == '':
            print(f"[{idx}/{len(new_papers)}] Searching PMID for: {paper['title']}")

            words = paper['title'].split()
            if len(words) < window_of_words_size:
                continue

            pmid_counts = {}

            for i in range(len(words) - window_of_words_size + 1):
                window_words = words[i:i+window_of_words_size]
                query = f"\"{' '.join(window_words)}\"[Title:~{proximity_deep_search}]"

                try:
                    handle = Entrez.esearch(db="pubmed", term=query, retmax=1)
                    record = Entrez.read(handle)
                    handle.close()
                except Exception as e:
                    print(f"Error during Entrez search for query '{query}': {e}")
                    continue

                id_list = record.get("IdList", [])
                if id_list:
                    for pmid in id_list:
                        pmid_counts[pmid] = pmid_counts.get(pmid, 0) + 1
                    print(f"Attempt {i+1}: Found PMIDs {id_list}")

                # Sleep after every n requests to keep rate limit
                # Use n > 3 ONLY if you have a PubMed api key
                if (i + 1) % limit_esearch_per_second == 0:
                    time.sleep(1)

            if pmid_counts:
                most_common_pmid = max(pmid_counts, key=pmid_counts.get)
                print(f"Most frequent PMID: {most_common_pmid}")
                paper["PMID"] = most_common_pmid

            print("Sleeping for 2 seconds to comply with Pubmed rate limit")
            time.sleep(2)

    print("Completed word window proximity search.")




if interactive_mode:
    # carry out the window_of_words search
    window_of_words_size = 5
    proximity_deep_search = 0
    limit_esearch_per_second = 3 # Use n > 3 ONLY if you have a PubMed api key

    word_window_proximity_search(
        new_papers,
        window_of_words_size,
        proximity_deep_search,
        limit_esearch_per_second
    )

    # add the Pubmed_title
    get_pubmed_titles(new_papers)
    display(pd.DataFrame(new_papers).style)


Starting word window proximity search for missing PMIDs (window size = 5, proximity = 0)...
[2/14] Searching PMID for: Regulation of Constitutive GPR3 Signaling and Surface Localization by GRK2 and b-arrestin-2 Overexpression in HEK293 Cells
Attempt 1: Found PMIDs ['23826079']
Attempt 2: Found PMIDs ['23826079']
Attempt 3: Found PMIDs ['23826079']
Attempt 4: Found PMIDs ['23826079']
Attempt 5: Found PMIDs ['23826079']
Attempt 6: Found PMIDs ['23826079']
Attempt 7: Found PMIDs ['23826079']
Most frequent PMID: 23826079
Sleeping for 2 seconds to comply with Pubmed rate limit
[3/14] Searching PMID for: A Role for Calcium Release-Activated Current CRAC in Cholinergic Modulation of Electrical Activity in Pancreatic 3-Cells
Attempt 1: Found PMIDs ['7647236']
Attempt 2: Found PMIDs ['7647236']
Attempt 3: Found PMIDs ['7647236']
Attempt 4: Found PMIDs ['11744714']
Attempt 5: Found PMIDs ['7647236']
Attempt 6: Found PMIDs ['7647236']
Attempt 7: Found PMIDs ['7647236']
Attempt 8: Found PMIDs ['76

,title,PMID,filename,Pubmed_title
0,RISC Assembly Defects in the Drosophila RNAi Mutant Armitage,15035985,1009Tomari2004.md,RISC assembly defects in the Drosophila RNAi mutant armitage.
1,Regulation of Constitutive GPR3 Signaling and Surface Localization by GRK2 and b-arrestin-2 Overexpression in HEK293 Cells,23826079,11Lowther2013.md,Regulation of Constitutive GPR3 Signaling and Surface Localization by GRK2 and β-arrestin-2 Overexpression in HEK293 Cells.
2,A Role for Calcium Release-Activated Current CRAC in Cholinergic Modulation of Electrical Activity in Pancreatic 3-Cells,7647236,3953Bertram1995.md,A role for calcium release-activated current (CRAC) in cholinergic modulation of electrical activity in pancreatic beta-cells.
3,News and Views,,114c.md,"Annals in Action: Your Newest Source for News, Views, and Reviews in Plastic Surgery."
4,"Crucial Role of Type 1, but Not Type 3, Inositol 1,4,5-Trisphosphate (IP3) Receptors in IP3-Induced Ca2 Release, Capacitative Ca2 Entry, and Proliferation of A7r5 Vascular Smooth Muscle Cells",11157673,1020.md,"Crucial role of type 1, but not type 3, inositol 1,4,5-trisphosphate (IP(3)) receptors in IP(3)-induced Ca(2+) release, capacitative Ca(2+) entry, and proliferation of A7r5 vascular smooth muscle cells."
5,Spreading silence with Sid,14759251,1011van_Roessel2004.md,Spreading silence with Sid.
6,The functional universe of membrane contact sites,31732717,Prinz2019The_functional_universe_of_membrane.md,The functional universe of membrane contact sites.
7,"Cytoplasmic Ca2+ oscillations evoked by receptor stimulation, G-protein activation, internal application of inositol trisphosphate or Ca2+: simultaneous microfluorimetry and Ca2+ dependent Cl- current recording in single pancreatic acinar cells",1690123,8.md,"Cytoplasmic Ca2+ oscillations evoked by receptor stimulation, G-protein activation, internal application of inositol trisphosphate or Ca2+: simultaneous microfluorimetry and Ca2+ dependent Cl- current recording in single pancreatic acinar cells."
8,Therapeutic silencing of an endogenous gene by systemic administration of modified siRNAs,15538359,1003Soutschek2004.md,Therapeutic silencing of an endogenous gene by systemic administration of modified siRNAs.
9,Animal virus replication and RNAi-mediated antiviral silencing in Caenorhabditis elegans,16107851,1000Lu2005.md,Animal virus replication and RNAi-mediated antiviral silencing in Caenorhabditis elegans.


## Compare llm-retrieved title with Pubmed_title with window_of_words similarity after deep title search. Remove PMID if similarity is < 0.94


In [30]:
# Compare llm-retrieved title with Pubmed_title with window_of_words similarity after deep title search.
# Remove PMID if similarity is < 0.94
def window_compare_titles_embeddings(new_papers, window_size=5, similarity_threshold=0.94):

    def get_windows(words, size):
        return [words[i:i+size] for i in range(len(words) - size + 1)] if len(words) >= size else []

    filtered = [p for p in new_papers if p.get('Pubmed_title')]

    if not filtered:
        print("No papers with 'Pubmed_title' to compare.")
        return pd.DataFrame()

    max_sims = []
    keep_flags = []
    for paper in filtered:
        title_words = paper['title'].split()
        pubmed_words = paper['Pubmed_title'].split()

        if len(title_words) < window_size or len(pubmed_words) < window_size:
            max_sim = 0
            # paper['PMID'] = ''  # commented out for testing
            keep = False
        else:
            title_windows = get_windows(title_words, window_size)
            pubmed_windows = get_windows(pubmed_words, window_size)

            similarities = []
            n = min(len(title_windows), len(pubmed_windows))
            for i in range(n):
                title_segment = " ".join(title_windows[i])
                pubmed_segment = " ".join(pubmed_windows[i])
                sim = batch_sentence_similarity([title_segment], [pubmed_segment])[0]
                similarities.append(sim)

            max_sim = max(similarities) if similarities else 0

            # Comment this 'if' for testing with different similarity thresholds
            if max_sim < similarity_threshold:
                paper['PMID'] = ''

            keep = max_sim >= similarity_threshold

        max_sims.append(max_sim)
        keep_flags.append(keep)

    df = pd.DataFrame({
        'title': [p['title'] for p in filtered],
        'Pubmed_title': [p.get('Pubmed_title', '') for p in filtered],
        'max_window_cosine_similarity': max_sims,
        'PMID': [p['PMID'] for p in filtered],
        'kept_based_on_threshold': keep_flags
    })

    return df




if interactive_mode:
  similarity_threshold_deep_search = 0.94
  df = window_compare_titles_embeddings(
      new_papers, window_size=5, similarity_threshold=similarity_threshold_deep_search
  )
  # display(pd.DataFrame(new_papers).style)
  display(df)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,title,Pubmed_title,max_window_cosine_similarity,PMID,kept_based_on_threshold
0,RISC Assembly Defects in the Drosophila RNAi M...,RISC assembly defects in the Drosophila RNAi m...,1.0,15035985,True
1,Regulation of Constitutive GPR3 Signaling and ...,Regulation of Constitutive GPR3 Signaling and ...,1.0,23826079,True
2,A Role for Calcium Release-Activated Current C...,A role for calcium release-activated current (...,1.0,7647236,True
3,News and Views,"Annals in Action: Your Newest Source for News,...",0.0,,False
4,"Crucial Role of Type 1, but Not Type 3, Inosit...","Crucial role of type 1, but not type 3, inosit...",1.0,11157673,True
5,Spreading silence with Sid,Spreading silence with Sid.,0.0,14759251,False
6,The functional universe of membrane contact sites,The functional universe of membrane contact si...,1.0,31732717,True
7,Cytoplasmic Ca2+ oscillations evoked by recept...,Cytoplasmic Ca2+ oscillations evoked by recept...,1.0,1690123,True
8,Therapeutic silencing of an endogenous gene by...,Therapeutic silencing of an endogenous gene by...,1.0,15538359,True
9,Animal virus replication and RNAi-mediated ant...,Animal virus replication and RNAi-mediated ant...,1.0,16107851,True


## Add full citation

In [32]:
# Add full citation
def add_full_citation(new_papers, batch_size=400):
    papers_with_pmid = [paper for paper in new_papers if paper.get('PMID')]
    if not papers_with_pmid:
        print("No papers with PMIDs found for citation fetching.")
        return

    print(f"Fetching full citations for {len(papers_with_pmid)} papers...")

    for i in range(0, len(papers_with_pmid), batch_size):
        batch = papers_with_pmid[i:i + batch_size]
        pmids_string = ",".join([p['PMID'] for p in batch])

        try:
            handle = Entrez.efetch(db="pubmed", id=pmids_string, retmode="xml")
            records = Entrez.read(handle)
            handle.close()

            for paper, article in zip(batch, records['PubmedArticle']):
                try:
                    article_data = article['MedlineCitation']['Article']
                    journal_info = article_data.get('Journal', {})
                    pub_date = journal_info.get('JournalIssue', {}).get('PubDate', {})

                    title = article_data.get('ArticleTitle')
                    journal = journal_info.get('Title')
                    year = pub_date.get('Year')

                    authors = article_data.get('AuthorList', [])
                    first_author_surname = authors[0].get('LastName') if authors else None

                    full_citation = f'{first_author_surname} ({year}). _{title}_ {journal}'

                    paper['full_citation'] = full_citation
                    paper['weblink'] = f'https://pubmed.ncbi.nlm.nih.gov/{paper["PMID"]}'
                    paper['Pubmed_title'] = title
                except (KeyError, IndexError, TypeError) as e:
                    logging.error(f"Error processing article for PMID {paper['PMID']}: {e}")

            print(f"Processed citations for papers {i+1} to {min(i+batch_size, len(papers_with_pmid))}")

        except Exception as e:
            logging.error(f"Error during Entrez fetch for PMIDs batch starting at index {i}: {e}")


if interactive_mode:
  add_full_citation(new_papers)

Fetching full citations for 13 papers...
Processed citations for papers 1 to 13


## Insert new papers in the citations.db

In [33]:
# Insert new papers in the citations.db (in batches of 100)
def insert_new_papers(new_papers, conn, batch_size=100):
    cursor = conn.cursor()
    total = len(new_papers)
    print(f"Ingesting {total} new papers into the database in batches of {batch_size}...")

    for idx, paper in enumerate(new_papers, 1):
        try:
            title = paper.get('title')
            PMID = paper.get('PMID') or None
            filename = paper.get('filename')
            full_citation = paper.get('full_citation')
            weblink = paper.get('weblink')
            Pubmed_title = paper.get('Pubmed_title')

            cursor.execute('''
                INSERT INTO citations (title, PMID, filename, full_citation, weblink, Pubmed_title)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (title, PMID, filename, full_citation, weblink, Pubmed_title))

            if idx % batch_size == 0 or idx == total:
                conn.commit()
                print(f"Committed batch up to paper {idx}/{total}")

        except Exception as e:
            logging.error(f"Failed to insert paper '{filename}': {e}")
            continue


if interactive_mode:
  insert_new_papers(new_papers, conn)

Ingesting 14 new papers into the database in batches of 100...
Committed batch up to paper 14/14


# Main

In [ ]:
%%time

def main():
    print("Starting citation ingestion process...\n")

    conn = None
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_filename)

        # Ensure citations table exists
        create_citations_db_if_not_existing(conn)

        # Extract new papers from markdown files not yet in the DB
        new_papers = get_new_papers_filename(md_folder, n_lines, conn)

        if not new_papers:
            print("No new markdown files found. Exiting.")
            return

        # Extract titles from the markdown files using an LLM
        llm_extract_titles(new_papers, batch_size=10)

        # Log any duplicate titles that already exist in the database
        log_duplicate_titles(new_papers, conn)


        # ---------- Initial PMID search with proximity = 2 --------------------
        proximity_initial_search = 2
        limit_esearch_per_second = 3 # Use n > 3 ONLY if you have a PubMed api key

        fetch_PMIDs_with_rate_limit(
            new_papers,
            proximity = proximity_initial_search,
            limit_esearch_per_second = limit_esearch_per_second
        )

        # Get the Pubmed_title for the found PMID
        get_pubmed_titles(new_papers)


        # Verify after INITIAL SEARCH that the similarity between llm-extracted
        # and efetch-ed Pubmed_title is > 0.94 otherwise remove PMID
        similarity_threshold_initial_search = 0.94

        df_initial_search = compare_titles_embeddings(
            new_papers,
            similarity_threshold=similarity_threshold_initial_search
        )



        # ---------- Deep search of PMID with strict proximity -----------------
        # Carry out the window_of_words search
        window_of_words_size = 5
        proximity_deep_search = 0
        limit_esearch_per_second = 3 # Use n > 3 ONLY if you have a PubMed api key

        word_window_proximity_search(
            new_papers,
            window_of_words_size,
            proximity_deep_search,
            limit_esearch_per_second
        )


        # add the Pubmed_title
        get_pubmed_titles(new_papers)


        # Verify after DEEP SEARCH that the similarity between llm-extracted
        # and efetch-ed Pubmed_title is > 0.94 otherwise remove PMID
        similarity_threshold_deep_search = 0.94

        df_deep_search = window_compare_titles_embeddings(
            new_papers, window_size=5, similarity_threshold=similarity_threshold_deep_search
        )




        # Fetch and store full citation details for validated PMIDs
        add_full_citation(new_papers)

        # Insert the newly processed papers into the citations database
        insert_new_papers(new_papers, conn)

        print("\nCitation ingestion process completed.")

        # Print summary of inserted records
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM citations")
        total = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM citations WHERE PMID != ''")
        with_pmid = cursor.fetchone()[0]

        without_pmid = total - with_pmid
        percentage_with = (with_pmid / total) * 100 if total else 0

        print(f"\nSummary of inserted records:")
        print(f"- Total records: {total}")
        print(f"- With PMID: {with_pmid}")
        print(f"- Without PMID: {without_pmid}")
        print(f"- Percentage with PMID: {percentage_with:.2f}%")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        if conn:
            conn.close()


main()


# Checking the results

In [44]:

import sqlite3
import pandas as pd

db_filename = "citations.db"

conn = sqlite3.connect(db_filename)
df = pd.read_sql_query("SELECT * FROM citations", conn)
conn.close()

df.style

count_no_pmid = df['PMID'].isna().sum()
print(f"Number of records without PMID: {count_no_pmid}")
print(df.count())

df
df.style
# df[df['PMID'].isna()]



# Quick fix for the api/insertion problem - **HANDLE WITH CARE**

When inserting large amount of papers (400+), for some reason one row might be skipped during the insertion of the PubMed title retrieved from the api. This results the llm-retrieved and the api-retrieved titles to be misaligned and provokes disasters when testing their similarity.

I still don't know what causes this, but for the moment the best is to just manually delete all the records in citations.db where PMID is None, and then rerun the script to search again for the articles where PMID was not found.



In [35]:
# import sqlite3

# # Path to your database
# db_path = 'citations.db'

# # Connect to the SQLite database
# conn = sqlite3.connect(db_path)
# cursor = conn.cursor()

# # Delete records where PMID is NULL or an empty string
# cursor.execute("DELETE FROM citations WHERE PMID IS NULL OR PMID = ''")

# # Commit changes and close the connection
# conn.commit()
# conn.close()

# print("Records with NULL or empty PMID removed.")
